In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('insurance_data_deep.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, train_size=0.8, random_state=42)

In [4]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [6]:
from tensorflow import keras
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=500)

Epoch 1/500


C:\Users\dhani\new_anaconda 3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 981ms/step - accuracy: 0.5000 - loss: 0.7428
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.5000 - loss: 0.7424
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.5000 - loss: 0.7420
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.5000 - loss: 0.7416
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5000 - loss: 0.7411
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.5000 - loss: 0.7407
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5000 - loss: 0.7403
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.5000 - loss: 0.7399
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - accuracy: 0.5000 - loss: 0.7395
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5000 - loss: 0.7390
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step - accuracy: 0.5000 - loss: 0.7386
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.5000 - loss: 0.7382

In [9]:
model.evaluate(X_test_scaled, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.8333 - loss: 0.5556


[0.5556402206420898, 0.8333333134651184]

In [10]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step


array([[0.67331934],
       [0.66144764],
       [0.67499775],
       [0.45126277],
       [0.60473907],
       [0.4531555 ]], dtype=float32)

In [11]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [13]:
coef, intercept = model.get_weights()
coef, intercept

(array([[0.76406646],
        [0.6513832 ]], dtype=float32),
 array([-0.3942271], dtype=float32))

In [14]:
def sigmoid(x):
    import math
    return 1/(1+math.exp(-x))

In [19]:
def pred_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

pred_function(.14,0)

C:\Users\dhani\AppData\Local\Temp\ipykernel_11484\1356804808.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1/(1+math.exp(-x))


0.4286753374869888

In [21]:
def sigmoid_numpy(X):
    return 1/(1+np.exp(-X))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [22]:
def log_loss(y_true, y_pred):
    epsilon = 1e-15
    y_pred_new = [max(i,epsilon) for i in y_pred]
    y_pred_new = [min(i, 1-epsilon) for i in y_pred_new]
    y_pred_new = np.array(y_pred_new)
    return -np.mean(y_true*np.log(y_pred_new) + (1-y_true)*np.log(1-y_pred_new))

In [29]:
def gradient_descent(age, affordibility, y_true, epochs):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_pred = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_pred)

        w1d = (1/n)*np.dot(np.transpose(age),(y_pred-y_true))
        w2d = (1/n)*np.dot(np.transpose(affordibility), (y_pred-y_true))

        bias_d = np.mean(y_pred - y_true)
        w1 = w1 - rate*w1d
        w2 = w2 - rate*w2d
        bias = bias - rate*bias_d

        print(f'Epochs: {i}, w1: {w1}, w2: {w2}, bias: {bias}, loss: {loss}')

    return w1, w2, bias

In [30]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000)

Epochs: 0, w1: 0.9736899318847281, w2: 0.931388810977659, bias: -0.11748951666770448, loss: 0.7428288579142563
Epochs: 1, w1: 0.9536535852311094, w2: 0.8740290167758512, bias: -0.21881533456146035, loss: 0.7072146449948487
Epochs: 2, w1: 0.9393731039296969, w2: 0.8271852202997496, bias: -0.3053620401943441, loss: 0.6814881914786812
Epochs: 3, w1: 0.9301932588998061, w2: 0.7897792032048467, bias: -0.37884372361582785, loss: 0.6633428084673968
Epochs: 4, w1: 0.9254091137248938, w2: 0.7605726653866934, bias: -0.44108236820018304, loss: 0.650742850709519
Epochs: 5, w1: 0.9243325693598607, w2: 0.738313053647322, bias: -0.49384257986251556, loss: 0.6420508089402462
Epochs: 6, w1: 0.926333296357235, w2: 0.7218280753843739, bias: -0.5387319906498417, loss: 0.6360356979531208
Epochs: 7, w1: 0.930858097563688, w2: 0.7100747303660235, bias: -0.5771558825717441, loss: 0.631816485354411
Epochs: 8, w1: 0.9374354910317362, w2: 0.7021560855322683, bias: -0.6103083840841516, loss: 0.6287844495353145
Ep

(np.float64(8.40135186247223),
 np.float64(1.5008804215445),
 np.float64(-4.382056927000267))